## Distributed Processing 1

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d mrmorj/hate-speech-and-offensive-language-dataset

Dataset URL: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset
License(s): CC0-1.0
  0% 0.00/1.01M [00:00<?, ?B/s]
100% 1.01M/1.01M [00:00<00:00, 140MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/hate-speech-and-offensive-language-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/labeled_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df = df.head(1000)

In [ ]:
df['tweet'][0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [ ]:
df.shape

(1000, 7)

### Data Preprocessing

In [ ]:
df['tweet'][2]

# We can see that our text contains punctuations, brackets, HTML tags and numbers
# We will preprocess this text in the next section

'!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit'

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase, and removes words starting with @ and RT'''

    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove RT and words starting with @
    sentence = re.sub(r'\brt\b', '', sentence)
    sentence = re.sub(r'@\w+', '', sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [ ]:
df["tweet"] = df['tweet'].apply(preprocess_text)

In [ ]:
df['tweet'][2]
# before preprocessing

' dawg ever fuck bitch start cry confused shit'

In [ ]:
df["class"] = df["class"].map({0:1,1:0,2:0})
y = df["class"]
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,0,woman complain cleaning house amp man always ...
1,1,3,0,3,0,0,boy dats cold tyga dwn bad cuffin dat hoe st ...
2,2,3,0,3,0,0,dawg ever fuck bitch start cry confused shit
3,3,3,0,2,1,0,look like tranny
4,4,6,0,6,0,0,shit hear might true might faker bitch told ya


In [ ]:
X = df["tweet"]
y = df["class"]

In [ ]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import gc

In [ ]:
# Preprocess text using RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def preprocess_text_batch(text_batch):
    inputs = tokenizer.batch_encode_plus(text_batch, return_tensors='tf', padding='max_length', truncation=True, max_length=768)
    return inputs

# def preprocess_text_batch(text_batch):
#     inputs = tokenizer.batch_encode_plus(text_batch, return_tensors='tf', padding='max_length', truncation=True, max_length=384)
#     return inputs

# Preprocess train and test data in batches

def preprocess_data_in_batches(X_data, output_dir):
    batch_size = 8
    num_batches = len(X_data) // batch_size + 1
    X_embeddings = []

    for i in range(num_batches):
        print(f"This is: {i+1}")
        # Clear GPU memory
        tf.keras.backend.clear_session()
        gc.collect()

        batch_text = X_data[i * batch_size : (i + 1) * batch_size]

        if not batch_text.empty:  # Check if batch_text is not empty
            batch_inputs = preprocess_text_batch(batch_text)
            batch_embeddings = roberta_model(batch_inputs)['last_hidden_state']
            X_embeddings.append(batch_embeddings)

            # Explicitly release GPU memory
            del batch_text
            del batch_inputs
            del batch_embeddings
            gc.collect()

            # Concatenate embeddings if there are enough batches
            if len(X_embeddings) >= 10:
                X_embeddings_concat = tf.concat(X_embeddings, axis=0)
                X_embeddings = [X_embeddings_concat]

    # Concatenate all batches for this chunk
    X_embeddings = np.concatenate(X_embeddings, axis=0)

    # Save embeddings and labels for this chunk
    embeddings_file = f"{output_dir}/X_embeddings_chunk_1.npy"
    np.save(embeddings_file, X_embeddings)

    print(f"Saved chunk 1")

    # return tf.concat(X_embeddings, axis=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Load RoBERTa model
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

output_dir = '/content/drive/MyDrive/Distributed Embeddings'

# Get RoBERTa embeddings for train and test data
preprocess_data_in_batches(X, output_dir)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

This is: 1
This is: 2
This is: 3
This is: 4
This is: 5
This is: 6
This is: 7
This is: 8
This is: 9
This is: 10
This is: 11
This is: 12
This is: 13
This is: 14
This is: 15
This is: 16
This is: 17
This is: 18
This is: 19
This is: 20
This is: 21
This is: 22
This is: 23
This is: 24
This is: 25
This is: 26
This is: 27
This is: 28
This is: 29
This is: 30
This is: 31
This is: 32
This is: 33
This is: 34
This is: 35
This is: 36
This is: 37
This is: 38
This is: 39
This is: 40
This is: 41
This is: 42
This is: 43
This is: 44
This is: 45
This is: 46
This is: 47
This is: 48
This is: 49
This is: 50
This is: 51
This is: 52
This is: 53
This is: 54
This is: 55
This is: 56
This is: 57
This is: 58
This is: 59
This is: 60
This is: 61
This is: 62
This is: 63
This is: 64
This is: 65
This is: 66
This is: 67
This is: 68
This is: 69
This is: 70
This is: 71
This is: 72
This is: 73
This is: 74
This is: 75
This is: 76
This is: 77
This is: 78
This is: 79
This is: 80
This is: 81
This is: 82
This is: 83
This is: 84
T

In [ ]:
y = y.values

In [ ]:
labels_file = f"{output_dir}/y_chunk_1.npy"
np.save(labels_file, y)
print(f"Saved chunk 1")

Saved chunk 1


## Distributed Processing 2

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d mrmorj/hate-speech-and-offensive-language-dataset

Dataset URL: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset
License(s): CC0-1.0
 99% 1.00M/1.01M [00:00<00:00, 2.70MB/s]
100% 1.01M/1.01M [00:00<00:00, 2.71MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/hate-speech-and-offensive-language-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/labeled_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df.shape

(24783, 7)

In [ ]:
# df = df.head(2000)
df = df.iloc[1000:2000]

In [ ]:
df.shape

(1000, 7)

### Data Preprocessing

In [ ]:
df['tweet'][1001]

# We can see that our text contains punctuations, brackets, HTML tags and numbers
# We will preprocess this text in the next section

'&#128514;&#128514;&#128514;&#128514;&#128514;&#128514;&#128514;&#128514;"@AlisaRenee: you a faggot if you eat pork chops."'

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase, and removes words starting with @ and RT'''

    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove RT and words starting with @
    sentence = re.sub(r'\brt\b', '', sentence)
    sentence = re.sub(r'@\w+', '', sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [ ]:
df["tweet"] = df['tweet'].apply(preprocess_text)

In [ ]:
df['tweet'][1001]
# before preprocessing

' faggot eat pork chops '

In [ ]:
df["class"] = df["class"].map({0:1,1:0,2:0})
y = df["class"]
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
1000,1022,3,0,3,0,0,pussy imma assume baby like day
1001,1023,3,1,2,0,0,faggot eat pork chops
1002,1024,3,0,3,0,0,mcnair brisket sizzlin grill side hoe laid uz...
1003,1025,3,1,2,0,0,mexican bitches shaped like silly puddy
1004,1026,3,1,2,0,0,hoe ya grandma got tattoo


In [ ]:
X = df["tweet"]
y = df["class"]

In [ ]:
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import gc

In [ ]:
# Preprocess text using RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def preprocess_text_batch(text_batch):
    inputs = tokenizer.batch_encode_plus(text_batch, return_tensors='tf', padding='max_length', truncation=True, max_length=768)
    return inputs

# def preprocess_text_batch(text_batch):
#     inputs = tokenizer.batch_encode_plus(text_batch, return_tensors='tf', padding='max_length', truncation=True, max_length=384)
#     return inputs

# Preprocess train and test data in batches

def preprocess_data_in_batches(X_data, output_dir):
    batch_size = 8
    num_batches = len(X_data) // batch_size + 1
    X_embeddings = []

    for i in range(num_batches):
        print(f"This is: {i+1}")
        # Clear GPU memory
        tf.keras.backend.clear_session()
        gc.collect()

        batch_text = X_data[i * batch_size : (i + 1) * batch_size]

        if not batch_text.empty:  # Check if batch_text is not empty
            batch_inputs = preprocess_text_batch(batch_text)
            batch_embeddings = roberta_model(batch_inputs)['last_hidden_state']
            X_embeddings.append(batch_embeddings)

            # Explicitly release GPU memory
            del batch_text
            del batch_inputs
            del batch_embeddings
            gc.collect()

            # Concatenate embeddings if there are enough batches
            if len(X_embeddings) >= 10:
                X_embeddings_concat = tf.concat(X_embeddings, axis=0)
                X_embeddings = [X_embeddings_concat]

    # Concatenate all batches for this chunk
    X_embeddings = np.concatenate(X_embeddings, axis=0)

    # Save embeddings and labels for this chunk
    embeddings_file = f"{output_dir}/X_embeddings_chunk_2.npy"
    np.save(embeddings_file, X_embeddings)

    print(f"Saved chunk 2")

    # return tf.concat(X_embeddings, axis=0)

In [ ]:
# Load RoBERTa model
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

output_dir = '/content/drive/MyDrive/Distributed Embeddings'

# Get RoBERTa embeddings for train and test data
preprocess_data_in_batches(X, output_dir)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

This is: 1
This is: 2
This is: 3
This is: 4
This is: 5
This is: 6
This is: 7
This is: 8
This is: 9
This is: 10
This is: 11
This is: 12
This is: 13
This is: 14
This is: 15
This is: 16
This is: 17
This is: 18
This is: 19
This is: 20
This is: 21
This is: 22
This is: 23
This is: 24
This is: 25
This is: 26
This is: 27
This is: 28
This is: 29
This is: 30
This is: 31
This is: 32
This is: 33
This is: 34
This is: 35
This is: 36
This is: 37
This is: 38
This is: 39
This is: 40
This is: 41
This is: 42
This is: 43
This is: 44
This is: 45
This is: 46
This is: 47
This is: 48
This is: 49
This is: 50
This is: 51
This is: 52
This is: 53
This is: 54
This is: 55
This is: 56
This is: 57
This is: 58
This is: 59
This is: 60
This is: 61
This is: 62
This is: 63
This is: 64
This is: 65
This is: 66
This is: 67
This is: 68
This is: 69
This is: 70
This is: 71
This is: 72
This is: 73
This is: 74
This is: 75
This is: 76
This is: 77
This is: 78
This is: 79
This is: 80
This is: 81
This is: 82
This is: 83
This is: 84
T

In [ ]:
y = y.values

In [ ]:
labels_file = f"{output_dir}/y_chunk_2.npy"
np.save(labels_file, y)
print(f"Saved chunk 2")

Saved chunk 2


### MaSTER

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os
import pandas as pd
from numpy import array
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Load preprocessed data from all batches
def load_preprocessed_data(num_batches, base_path):
    X_list = []
    y_list = []
    for i in range(1, num_batches + 1):
        X_batch = np.load(os.path.join(base_path, f'X_embeddings_chunk_{i}.npy'))
        y_batch = np.load(os.path.join(base_path, f'y_chunk_{i}.npy'))
        X_list.append(X_batch)
        y_list.append(y_batch)
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    return X, y

base_path = '/content/drive/MyDrive/Distributed Embeddings'
num_batches = 2  # Adjust based on the number of batches you have
X_train_embeddings, y_train = load_preprocessed_data(num_batches, base_path)

# Expand dimensions for LSTM input (timesteps dimension)
# X_train_embeddings = np.expand_dims(X_train_embeddings, axis=1)




In [2]:
# Split into training and validation sets
X_train_embeddings, X_val_embeddings, y_train, y_val = train_test_split(X_train_embeddings, y_train, test_size=0.2, random_state=42)

In [3]:
X_train_embeddings.shape

(1600, 768, 768)

In [4]:
# Build RoBERTa-LSTM model
def build_roberta_lstm_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(None, 768)))
    model.add(Dense(1, activation='sigmoid'))
    return model

roberta_lstm_model = build_roberta_lstm_model()

# Compile model
# roberta_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
# Compile model
roberta_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train model
# roberta_lstm_history = roberta_lstm_model.fit(X_train_embeddings, y_train, batch_size=32, epochs=5, validation_data=(X_val_embeddings, y_val))
roberta_lstm_history = roberta_lstm_model.fit(X_train_embeddings, y_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
40/40 [==============================] - 10s 140ms/step - loss: 0.2121 - accuracy: 0.9344 - val_loss: 0.2325 - val_accuracy: 0.9375
Epoch 2/5
40/40 [==============================] - 6s 144ms/step - loss: 0.1693 - accuracy: 0.9578 - val_loss: 0.2327 - val_accuracy: 0.9375
Epoch 3/5
40/40 [==============================] - 5s 121ms/step - loss: 0.1628 - accuracy: 0.9578 - val_loss: 0.2317 - val_accuracy: 0.9375
Epoch 4/5
40/40 [==============================] - 4s 91ms/step - loss: 0.1555 - accuracy: 0.9578 - val_loss: 0.2390 - val_accuracy: 0.9375
Epoch 5/5
40/40 [==============================] - 5s 136ms/step - loss: 0.1323 - accuracy: 0.9578 - val_loss: 0.2423 - val_accuracy: 0.9344


In [5]:
# Print training and validation accuracy
train_acc = roberta_lstm_history.history['accuracy'][-1]
val_acc = roberta_lstm_history.history['val_accuracy'][-1]
print(f"Training Accuracy: {train_acc:.3f}")
print(f"Validation Accuracy: {val_acc:.3f}")

Training Accuracy: 0.958
Validation Accuracy: 0.934
